In [1]:
import requests
import re

In [58]:
from HTMLParser import HTMLParser
class ParseOL(HTMLParser):
    
    def __init__(self):
        HTMLParser.__init__(self) #Old Style Class
        self.attributes = []
        
    def handle_starttag(self, tag, attrs):
        pass
    
    def handle_endtag(self, tag):
        pass
    
    def handle_data(self, data):
        self.attributes.append(data)

In [2]:
baseurl = 'http://www.newhavencrimelog.org'
r = requests.get('{}/cl'.format(baseurl))

In [3]:
if r.status_code == 200:
    headers = r.headers
    txt = r.content.split('\n')

In [4]:
pattern = re.compile(r'(\/cl\/types\/\d+\/[a-z]*[a-z\-]*\/)')
upper_pattern = re.compile(r'[A-Z]+[A-Z]')
searchurls = []
for l in txt:
    p = pattern.search(l)
    if p:
        searchurls.append(p.group())
        

In [5]:
print searchurls[0]

/cl/types/9/all-cases-dead-on-arrival/


In [6]:
def extract_markers(txt):
    pattern = re.compile(r'var markerList=')
    marker_str = ''
    searching = False
    for i, l in enumerate(txt):
        if 'markerList=' in l:
            start = i
            searching = True
        if searching:
            marker_str += l + '\n'
            if '];' in l:
                break
    return marker_str

In [133]:
crimes = {}
dsearch = re.compile(r'-\d+.\d+,\d+.\d+')
for cnt, url in enumerate(searchurls):
    #Setup the search URL and data key
    leaf = '{}{}'.format(baseurl, url)
    k = leaf.split('/')[-2]
    crimes[k] = { "type": "FeatureCollection",
    "features": []}
    
    #Make the HTTP request
    r = requests.get('{}{}'.format(baseurl, url))
    txt = r.content.split('\n')
    
    #Grab the markers tag from the map javascript
    marker_str = extract_markers(txt)
    
    i = 0
    for l in marker_str.split('\n'):
        #Found coordinates...
        if '[ -' in l:

            #Regex to grab coordinates
            coords = map(float,dsearch.search(l).group().split(','))
            
            if len(coords) > 0:
                #Create an HTML parser
                parser = ParseOL()
                
                #Setup the geojson response
                crimes[k]['features'].append({ "type": "Feature",
                                               "geometry": {"type": "Point", "coordinates": []},
                                               "properties": {}
                                            })
                cfeature = crimes[k]['features'][i]
                #Get the HTML out and parse the values
                parser.feed(l.split("'")[1])
                attributes = {}
                for a in parser.attributes:
                    if a[-1] == ':':
                        key = a[:-1]
                        attributes[key] = ''
                        collect = True
                        cstring = ''
                    else:
                        attributes[key] += ' {}'.format(a)

                cfeature['geometry']['coordinates'] = coords
                cfeature['properties'] = attributes
                i += 1
            else:
                print l

In [134]:
print crimes.keys()

['recovered-prop-owner-unk', 'burglary-schoolnonforc', 'theft-of-license-plates', 'suspicious-fires', 'stolen-auto-nhrecvd-ot', 'dwi-drugs', 'theft-of-lic-reg-or-title', 'robbery-attempt-oth-weap', 'burglary-other-forcible', 'incident-unfounded', 'recovered-auto-stolen-ot', 'field-interrogation', 'assault-wdangerous-weapon', 'falsely-reporting-incident', 'forgerychecks-mv-document', 'robbery-street-firearm', 'crim-mischief-pub-propsvc', 'supplementary-investigation', 'robbery-res-firearm', 'assault-with-a-firearm', 'other-alarms', 'theft-university-acadbldg', 'awol', 'mv-accident-nonfatal', 'robbery-attempt-no-weapon', 'landlordtenant-dispute', 'unable-to-locate-complainant', 'wanted-personnew-haven', 'cruelty-to-animals', 'burglary-commnonforcible', 'possessdangcontrol-drugs', 'licensereg-check', 'medical-assistance-rendered', 'robbery-street-no-weapon', 'theft-of-auto', 'other-investigation', 'stolen-other-nhrecvd-ot', 'possession-of-paraphernalia', 'burglary-resforcible', 'building-

In [135]:
crimes['sudden-deathbodies-found']

{'features': [{'geometry': {'coordinates': [-72.873112, 41.315345],
    'type': 'Point'},
   'properties': {'date': ' Wed, Sept. 3rd 2014',
    'intersections': ' HEMINGWAY/FOXON',
    'street': ' 321 EASTERN ST',
    'time': ' 04:54 p.m.'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-72.939412, 41.309609], 'type': 'Point'},
   'properties': {'date': ' Tue, Sept. 16th 2014',
    'intersections': ' GEORGE/CHAPEL',
    'street': ' 162 DAY ST',
    'time': ' 05:52 a.m.'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-72.8732, 41.315137], 'type': 'Point'},
   'properties': {'date': ' Thu, Sept. 18th 2014',
    'intersections': ' HEMINGWAY/FOXON',
    'street': ' 311 EASTERN ST',
    'time': ' 09:37 a.m.'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-72.931544, 41.311728], 'type': 'Point'},
   'properties': {'date': ' Sun, Sept. 21st 2014',
    'intersections': ' HOWE/INTERSECTION',
    'street': ' 100 BROADWAY ST',
    'time': ' 06:53 a.m.'},
   'type': 'F

In [136]:
import json
for k, v in crimes.iteritems():
    with open('new_haven_data/{}.json'.format(k), 'w') as f:
        json.dump(v, f)